# Layer API
The ncnn module contains the layer abstraction. 

In [2]:
import ncdl
import torch

import torch.nn as nn
import ncdl.nn as ncnn

from ncdl import Lattice, Stencil

We start by defining some data.

In [17]:
qc = Lattice("qc")

blue_coset = torch.rand(4, 4, 10, 10)
orange_coset = torch.rand(4, 4, 9, 9)

lt = qc(
    blue_coset, 
    orange_coset
) 

The following is a worked example of a residual block. The code is short and very pytorch-ic.

In [19]:
class QuincunxResBlock(nn.Module):
    def __init__(self, channels_in, channels_out, channels_mid):
        super().__init__()
        
        self.lattice = Lattice("qc")
        
        # Define a stencil
        stencil = Stencil([
            (0, 0), (2, 0), (0, 2), (2, 2), (1, 1)
        ], self.lattice, center=(1,1))
        
        
        # define the blocks
        self.inner_block = nn.Sequential(
            ncnn.LatticePad(self.lattice, stencil),
            ncnn.LatticeConvolution(self.lattice, channels_in, channels_mid, stencil),
            ncnn.LatticeBatchNorm(self.lattice, channels_mid),
            ncnn.ReLU(),
            
            ncnn.LatticePad(self.lattice, stencil),
            ncnn.LatticeConvolution(self.lattice, channels_mid, channels_in, stencil),
            ncnn.LatticeBatchNorm(self.lattice, channels_in),
            ncnn.ReLU()
        )
        
        self.outer_block = nn.Sequential(
            ncnn.LatticePad(self.lattice, stencil),
            ncnn.LatticeConvolution(self.lattice, channels_mid, channels_in, stencil),
            ncnn.LatticePad(self.lattice, stencil), #note, this should actually be -inf padded...
            ncnn.LatticeMaxPooling(self.lattice, stencil)
        )
    def forward(self, lt):
        lto = self.inner_block(lt) + lt
        return self.outer_block(lto)

qcrb = QuincunxResBlock(4, 8, 4)

Using this layer is exactly the same as in the pytorch layer api.

In [21]:
lto = qcrb(lt)